### Prerequisites

In [1]:
import requests

In [2]:
url = "https://raw.githubusercontent.com/mitre/cti/master/enterprise-attack/enterprise-attack.json"
response = requests.get(url)
data = response.json()

### Exploring the DataSet

In [52]:
print("Type of Data:", type(data))
print("Keys: ", list(data.keys()))
print("Number of Entries:", len(data["objects"]))
print(f"\nKeys of object:\n-----------------------------------------------")
for key in data["objects"][0].keys():
  k = key
  t = type(key)
  print(f"{k:<30} : {str(t):<20}")

Type of Data: <class 'dict'>
Keys:  ['type', 'id', 'objects', 'spec_version']
Number of Entries: 24771

Keys of object:
-----------------------------------------------
type                           : <class 'str'>       
id                             : <class 'str'>       
created                        : <class 'str'>       
created_by_ref                 : <class 'str'>       
external_references            : <class 'str'>       
object_marking_refs            : <class 'str'>       
modified                       : <class 'str'>       
name                           : <class 'str'>       
description                    : <class 'str'>       
tactic_refs                    : <class 'str'>       
x_mitre_modified_by_ref        : <class 'str'>       
x_mitre_deprecated             : <class 'str'>       
x_mitre_domains                : <class 'str'>       
x_mitre_version                : <class 'str'>       
x_mitre_attack_spec_version    : <class 'str'>       


In [54]:
techniques = []
for obj in data["objects"]:
  if obj["type"] == "attack-pattern":
    techniques.append(obj)
print(len(techniques)," objects of type - 'attack-pattern' found")

835  objects of type - 'attack-pattern' found


### Threat Scoring Logic

In [19]:
def threat_score(tech):
  score = 5
  name = tech["name"].lower()
  if "credential" in name:
    score += 3
  if "privilege" in name:
    score += 3
  if "execution" in name:
    score += 2
  if "persistence" in name:
    score += 2
  if "lateral" in name:
    score += 2

  desc = tech["description"].lower()
  if "adminstrator" in desc:
    score += 2
  if "remote" in desc:
    score += 2
  if "bypass" in desc:
    score += 2
  if "stealth" in desc:
    score += 1
  return score

### Assigning Scores

In [21]:
scored_techniques = []
for t in techniques:
  score = threat_score(t)
  scored_techniques.append((t["name"],score))

### Prioritizing Threats

In [58]:
scored_techniques = sorted(scored_techniques, key= lambda x: x[1], reverse=True)
print("TOP 10 High Risk Techniques")
print("---------------------------------------")
for i in range(10):
  print(f"{scored_techniques[i][0]:<30} : {scored_techniques[i][1]:>3}")


TOP 10 High Risk Techniques
---------------------------------------
Pass the Ticket                :  10
Credentials In Files           :  10
Additional Cloud Credentials   :  10
Forge Web Credentials          :  10
Credentials in Files           :  10
Credentials                    :  10
Pass the Hash                  :  10
Extra Window Memory Injection  :   9
PubPrn                         :   9
Steal Web Session Cookie       :   9


### Critical Threat Filter

In [40]:
print(f"Critical threats\n------------------------------------------------------------------------")
h1 = "Name of the Threat"
h2 = "Threat Score"
print(f"{h1:<50}      {h2}")
print("------------------------------------------------------------------------")
for t in scored_techniques:
  if(t[1]<8.9):
    break
  print(f"{t[0]:<50} ---> {t[1]:>8}")

Critical threats
------------------------------------------------------------------------
Name of the Threat                                      Threat Score
------------------------------------------------------------------------
Pass the Ticket                                    --->       10
Credentials In Files                               --->       10
Additional Cloud Credentials                       --->       10
Forge Web Credentials                              --->       10
Credentials in Files                               --->       10
Credentials                                        --->       10
Pass the Hash                                      --->       10
Extra Window Memory Injection                      --->        9
PubPrn                                             --->        9
Steal Web Session Cookie                           --->        9
Bypass User Account Control                        --->        9
Application Access Token                           --